In [1]:
import os
import glob
import itertools
import pickle

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
# Read in daily data
data_path = os.path.join('.', 'data')
daily_data_path = os.path.join(data_path, 'day_approach_maskedID_timeseries.csv')
weekly_data_path = os.path.join(data_path, 'week_approach_maskedID_timeseries.csv')

daily_df = pd.read_csv(daily_data_path)
daily_df.head()

,nr. sessions,total km,km Z3-4,km Z5-T1-T2,km sprinting,strength training,hours alternative,perceived exertion,perceived trainingSuccess,perceived recovery,nr. sessions.1,total km.1,km Z3-4.1,km Z5-T1-T2.1,km sprinting.1,strength training.1,hours alternative.1,perceived exertion.1,perceived trainingSuccess.1,perceived recovery.1,nr. sessions.2,total km.2,km Z3-4.2,km Z5-T1-T2.2,km sprinting.2,strength training.2,hours alternative.2,perceived exertion.2,perceived trainingSuccess.2,perceived recovery.2,nr. sessions.3,total km.3,km Z3-4.3,km Z5-T1-T2.3,km sprinting.3,strength training.3,hours alternative.3,perceived exertion.3,perceived trainingSuccess.3,perceived recovery.3,nr. sessions.4,total km.4,km Z3-4.4,km Z5-T1-T2.4,km sprinting.4,strength training.4,hours alternative.4,perceived exertion.4,perceived trainingSuccess.4,perceived recovery.4,nr. sessions.5,total km.5,km Z3-4.5,km Z5-T1-T2.5,km sprinting.5,strength training.5,hours alternative.5,perceived exertion.5,perceived trainingSuccess.5,perceived recovery.5,nr. sessions.6,total km.6,km Z3-4.6,km Z5-T1-T2.6,km sprinting.6,strength training.6,hours alternative.6,perceived exertion.6,perceived trainingSuccess.6,perceived recovery.6,Athlete ID,injury,Date
0,1.0,5.8,0.0,0.6,1.2,0.0,0.00,0.11,0.00,0.18,0.0,0.0,0.0,0.0,0.0,0.0,0.00,-0.01,-0.01,-0.01,1.0,0.0,0.0,0.0,0.0,1.0,0.00,0.10,0.00,0.17,0.0,0.0,0.0,0.0,0.0,0.0,0.00,-0.01,-0.01,-0.01,1.0,0.0,0.0,0.0,0.0,0.0,1.08,0.08,0.00,0.18,1.0,16.4,10.0,0.0,0.0,1.0,0.0,0.11,0.00,0.17,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.10,0.00,0.15,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.00,-0.01,-0.01,-0.01,1.0,0.0,0.0,0.0,0.0,1.0,0.00,0.10,0.00,0.17,0.0,0.0,0.0,0.0,0.0,0.0,0.00,-0.01,-0.01,-0.01,1.0,0.0,0.0,0.0,0.0,0.0,1.08,0.08,0.00,0.18,1.0,16.4,10.0,0.0,0.0,1.0,0.00,0.11,0.00,0.17,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.10,0.00,0.15,1.0,5.2,0.0,0.5,1.2,0.0,0.0,0.10,0.00,0.17,0,0,1
2,1.0,0.0,0.0,0.0,0.0,1.0,0.00,0.10,0.00,0.17,0.0,0.0,0.0,0.0,0.0,0.0,0.00,-0.01,-0.01,-0.01,1.0,0.0,0.0,0.0,0.0,0.0,1.08,0.08,0.00,0.18,1.0,16.4,10.0,0.0,0.0,1.0,0.00,0.11,0.00,0.17,1.0,0.0,0.0,0.0,0.0,0.0,1.00,0.10,0.00,0.15,1.0,5.2,0.0,0.5,1.2,0.0,0.0,0.10,0.00,0.17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.01,-0.01,-0.01,0,0,2
3,0.0,0.0,0.0,0.0,0.0,0.0,0.00,-0.01,-0.01,-0.01,1.0,0.0,0.0,0.0,0.0,0.0,1.08,0.08,0.00,0.18,1.0,16.4,10.0,0.0,0.0,1.0,0.00,0.11,0.00,0.17,1.0,0.0,0.0,0.0,0.0,0.0,1.00,0.10,0.00,0.15,1.0,5.2,0.0,0.5,1.2,0.0,0.00,0.10,0.00,0.17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.01,-0.01,-0.01,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.10,0.00,0.17,0,0,3
4,1.0,0.0,0.0,0.0,0.0,0.0,1.08,0.08,0.00,0.18,1.0,16.4,10.0,0.0,0.0,1.0,0.00,0.11,0.00,0.17,1.0,0.0,0.0,0.0,0.0,0.0,1.00,0.10,0.00,0.15,1.0,5.2,0.0,0.5,1.2,0.0,0.00,0.10,0.00,0.17,0.0,0.0,0.0,0.0,0.0,0.0,0.00,-0.01,-0.01,-0.01,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.10,0.00,0.17,1.0,17.6,7.2,0.0,0.0,0.0,0.0,0.11,0.00,0.17,0,0,4


In [3]:
# Create datasets
target_col = 'injury'
feat_cols = [c for c in daily_df.columns if c not in ['Athlete ID', 'injury', 'Date']]

X = daily_df[feat_cols].values
y = daily_df[target_col].values

In [4]:
# Create functions for cross validation - ensure train/test splits preserve class imbalance of full dataset
from sklearn.model_selection import cross_validate, StratifiedKFold

def run_cv(model, X, y, scoring, n_splits, verbose=0, return_estimators=False):
    cv_results = cross_validate(model, X, y, scoring=scoring, cv=StratifiedKFold(n_splits=n_splits), verbose=verbose, return_estimator=return_estimators)
    return cv_results

def display_results(results_dict):
    for k in results_dict:
        if isinstance(results_dict[k], type(np.array([]))):
            print(f"Average {k.capitalize()}: {results_dict[k].mean():04f}")
            print()

## Linear regression

In [5]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
cv_scores = run_cv(lin_reg, X, y, scoring='neg_root_mean_squared_error', n_splits=5, verbose=0, return_estimators=True)
display_results(cv_scores)

Average Fit_time: 0.114480

Average Score_time: 0.002161

Average Test_score: -0.116197



In [6]:
# Examine coefficients of each estimator
est_coefs = []
for i, fc in enumerate(feat_cols):
    fc_coefs = pd.Series([est.coef_[i] for est in cv_scores['estimator']], name=fc)
    est_coefs.append(fc_coefs)
coefs_df = pd.concat(est_coefs, axis=1)

coefs_df.describe()

,nr. sessions,total km,km Z3-4,km Z5-T1-T2,km sprinting,strength training,hours alternative,perceived exertion,perceived trainingSuccess,perceived recovery,nr. sessions.1,total km.1,km Z3-4.1,km Z5-T1-T2.1,km sprinting.1,strength training.1,hours alternative.1,perceived exertion.1,perceived trainingSuccess.1,perceived recovery.1,nr. sessions.2,total km.2,km Z3-4.2,km Z5-T1-T2.2,km sprinting.2,strength training.2,hours alternative.2,perceived exertion.2,perceived trainingSuccess.2,perceived recovery.2,nr. sessions.3,total km.3,km Z3-4.3,km Z5-T1-T2.3,km sprinting.3,strength training.3,hours alternative.3,perceived exertion.3,perceived trainingSuccess.3,perceived recovery.3,nr. sessions.4,total km.4,km Z3-4.4,km Z5-T1-T2.4,km sprinting.4,strength training.4,hours alternative.4,perceived exertion.4,perceived trainingSuccess.4,perceived recovery.4,nr. sessions.5,total km.5,km Z3-4.5,km Z5-T1-T2.5,km sprinting.5,strength training.5,hours alternative.5,perceived exertion.5,perceived trainingSuccess.5,perceived recovery.5,nr. sessions.6,total km.6,km Z3-4.6,km Z5-T1-T2.6,km sprinting.6,strength training.6,hours alternative.6,perceived exertion.6,perceived trainingSuccess.6,perceived recovery.6
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000e+00,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,0.002764,-0.000033,-0.000222,0.000437,0.001730,-0.000733,-0.003400,0.003084,0.000587,0.000606,-0.001330,-0.000264,0.000256,0.000214,0.001498,0.006625,0.001319,-0.000495,0.008083,-0.009135,0.002074,-0.000310,0.000910,0.000286,0.002559,0.000934,-0.001048,0.003088,-0.002426,0.001680,-0.001216,-0.000148,0.000250,0.000432,0.001676,0.001128,-0.000878,0.006421,0.005077,-0.003820,0.003240,-0.000229,0.000498,0.000740,-0.000868,-0.002946,-0.002399,0.003571,0.002928,0.000199,-0.000974,5.880961e-05,0.000891,0.001133,0.000882,-0.001077,-0.001818,0.005620,-0.005537,0.006350,0.002400,-0.000159,0.000519,0.000878,0.003687,0.000205,0.002975,0.004123,-0.007136,0.004208
std,0.001498,0.000148,0.000157,0.000296,0.001621,0.001753,0.000708,0.002368,0.000567,0.005019,0.001184,0.000087,0.000141,0.000195,0.002316,0.002164,0.000717,0.002859,0.001933,0.002509,0.001307,0.000059,0.000071,0.000251,0.002409,0.002113,0.000767,0.002253,0.001819,0.003970,0.000602,0.000101,0.000140,0.000175,0.002541,0.000398,0.000480,0.004412,0.001597,0.004227,0.000699,0.000049,0.000128,0.000146,0.000733,0.001121,0.000590,0.001303,0.001396,0.004502,0.000793,1.351316e-04,0.000227,0.000170,0.000389,0.001269,0.000821,0.003115,0.002684,0.003749,0.001532,0.000110,0.000102,0.000215,0.002575,0.002007,0.001999,0.001789,0.001055,0.000555
min,0.000732,-0.000179,-0.000455,-0.000084,0.000112,-0.002698,-0.004025,0.001095,-0.000376,-0.005039,-0.002682,-0.000359,0.000018,-0.000006,-0.000380,0.003297,0.000190,-0.004710,0.005496,-0.013030,0.000796,-0.000384,0.000832,-0.000077,0.000256,-0.001401,-0.002291,0.001270,-0.004241,-0.004136,-0.002108,-0.000253,0.000117,0.000132,-0.000837,0.000763,-0.001309,0.002648,0.003014,-0.008071,0.002829,-0.000272,0.000388,0.000597,-0.001551,-0.004029,-0.003218,0.002342,0.000435,-0.005205,-0.001830,-4.041327e-05,0.000676,0.000953,0.000294,-0.003268,-0.002980,0.002468,-0.008502,0.003291,0.000075,-0.000326,0.000395,0.000527,0.002060,-0.003161,0.000083,0.001863,-0.008480,0.003758
25%,0.001709,-0.000169,-0.000314,0.000468,0.000860,-0.001248,-0.003989,0.002168,0.000547,-0.001464,-0.002325,-0.000335,0.000237,0.000027,0.000153,0.005987,0.001237,-0.001274,0.0

## Logistic Regression

In [7]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(solver='lbfgs', max_iter=1000)
cv_scores = run_cv(log_reg, X, y, scoring='neg_root_mean_squared_error', n_splits=5, verbose=0, return_estimators=True)
display_results(cv_scores)

Average Fit_time: 2.041666

Average Score_time: 0.001622

Average Test_score: -0.121144



In [8]:
# Attempt grid search to tune logistic regression model
from sklearn.model_selection import GridSearchCV
from scipy.stats import uniform

def grid_search_cv(est, param_grid, scoring, n_splits=5):
    return GridSearchCV(est, param_grid=param_grid, scoring=scoring, cv=StratifiedKFold(n_splits=n_splits))

def print_grid_search_scores(gs_cv_obj, param_grid):
    # Find non-singleton elements of param_grid
    ns_grid_params = [k for k in param_grid if len(param_grid[k]) > 1]

    # Print mean test score for each parameter combination
    if len(ns_grid_params) == 1:
        index = [f"{ns_grid_params[0]}_{p}" for p in param_grid[ns_grid_params[0]]]
        name = 'mean_test_root_mean_squared_error'
        print(pd.DataFrame(gs_cv_obj.cv_results_['mean_test_score'], index=index, columns=[name]))
    elif len(ns_grid_params) == 2:
        idx_p = ns_grid_params[0]
        col_p = ns_grid_params[1]
        index = [f"{idx_p}_{p}" for p in param_grid[idx_p]]
        columns = [f"{col_p}_{p}" for p in param_grid[col_p]]
        print(pd.DataFrame(gs_cv_obj.cv_results_['mean_test_score'].reshape(len(index), len(columns)), index=index, columns=columns))

In [17]:
# No penalty
log_reg = LogisticRegression()
param_grid = {'penalty': ['none'],
              'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'],
              'max_iter': [1000]}

gs_cv_no_penalty = grid_search_cv(log_reg, param_grid, scoring='neg_root_mean_squared_error', n_splits=5)
gs_cv_no_penalty.fit(X, y)

/Users/davidchung/opt/anaconda3/envs/runner-preds/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/davidchung/opt/anaconda3/envs/runner-preds/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/davidchung/opt/anaconda3/envs/runner-preds/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/davidchung/opt/anaconda3/envs/runner-preds/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which 

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=LogisticRegression(),
             param_grid={'max_iter': [1000], 'penalty': ['none'],
                         'solver': ['newton-cg', 'lbfgs', 'sag', 'saga']},
             scoring='neg_root_mean_squared_error')

In [67]:
# Print results of training logistic regression w/ no penalty
param_grid = {'penalty': ['none'],
              'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'],
              'max_iter': [1000]}
print_grid_search_scores(gs_cv_no_penalty, param_grid)

                  mean_test_root_mean_squared_error
solver_newton-cg                          -0.121228
solver_lbfgs                              -0.121228
solver_sag                                -0.121144
solver_saga                               -0.121144


In [43]:
# Save pickle object
with open('./test/log_reg_gs_no_penalty.pkl', 'wb') as f:
    pickle.dump(gs_cv_no_penalty, f)

In [34]:
# Grid search w/ L2 penalty
param_grid = {'penalty' : ['l2'],
              'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
              'C': [0.01, 0.02, 0.04, 0.1, 0.2, 0.4, 1.0, 2.0, 4.0, 10.],
              'max_iter': [1000],
              }
gs_cv_l2 = grid_search_cv(LogisticRegression(), param_grid, scoring='neg_root_mean_squared_error', n_splits=5)
gs_cv_l2.fit(X, y)

/Users/davidchung/opt/anaconda3/envs/runner-preds/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/davidchung/opt/anaconda3/envs/runner-preds/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/davidchung/opt/anaconda3/envs/runner-preds/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/davidchung/opt/anaconda3/envs/runner-preds/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which 

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=LogisticRegression(),
             param_grid={'C': [0.01, 0.02, 0.04, 0.1, 0.2, 0.4, 1.0, 2.0, 4.0,
                               10.0],
                         'max_iter': [1000], 'penalty': ['l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             scoring='neg_root_mean_squared_error')

In [45]:
# Save pickle object
with open('./test/log_reg_gs_l2.pkl', 'wb') as f:
    pickle.dump(gs_cv_l2, f)

In [68]:
# Print results of training logistic regression w/ L2 penalty
param_grid = {'penalty' : ['l2'],
              'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
              'C': [0.01, 0.02, 0.04, 0.1, 0.2, 0.4, 1.0, 2.0, 4.0, 10.],
              'max_iter': [1000],
              }
print_grid_search_scores(gs_cv_l2, param_grid)

                    C_0.01    C_0.02    C_0.04     C_0.1     C_0.2     C_0.4     C_1.0     C_2.0     C_4.0    C_10.0
solver_newton-cg -0.119053 -0.119053 -0.118405 -0.119053 -0.119053 -0.120215 -0.120302 -0.120302 -0.120215 -0.120215
solver_lbfgs     -0.121075 -0.121075 -0.121244 -0.121075 -0.121075 -0.121328 -0.121328 -0.121661 -0.121328 -0.121328
solver_liblinear -0.121228 -0.121228 -0.121578 -0.121228 -0.121228 -0.121228 -0.121228 -0.121479 -0.121228 -0.121228
solver_sag       -0.121228 -0.121144 -0.121312 -0.121228 -0.121228 -0.121144 -0.121144 -0.121228 -0.121144 -0.121228
solver_saga      -0.121144 -0.121228 -0.121228 -0.121144 -0.121228 -0.121228 -0.121228 -0.121228 -0.121144 -0.121144


In [10]:
# Grid search w/ L1 penalty
param_grid = {'penalty' : ['l1'],
              'solver': ['liblinear', 'saga'],
              'C': [0.01, 0.02, 0.04, 0.1, 0.2, 0.4, 1.0, 2.0, 4.0, 10.],
              'max_iter': [1000],
              }
gs_cv_l1 = grid_search_cv(LogisticRegression(), param_grid, scoring='neg_root_mean_squared_error', n_splits=5)
gs_cv_l1.fit(X, y)

/Users/davidchung/opt/anaconda3/envs/runner-preds/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/davidchung/opt/anaconda3/envs/runner-preds/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/davidchung/opt/anaconda3/envs/runner-preds/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/davidchung/opt/anaconda3/envs/runner-preds/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which 

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=LogisticRegression(),
             param_grid={'C': [0.01, 0.02, 0.04, 0.1, 0.2, 0.4, 1.0, 2.0, 4.0,
                               10.0],
                         'max_iter': [1000], 'penalty': ['l1'],
                         'solver': ['liblinear', 'saga']},
             scoring='neg_root_mean_squared_error')

In [11]:
# Print results of training logistic regression w/ L1 penalty
param_grid = {'penalty' : ['l1'],
              'solver': ['liblinear', 'saga'],
              'C': [0.01, 0.02, 0.04, 0.1, 0.2, 0.4, 1.0, 2.0, 4.0, 10.],
              'max_iter': [1000],
              }
print_grid_search_scores(gs_cv_l1, param_grid)

                    C_0.01    C_0.02    C_0.04     C_0.1     C_0.2     C_0.4     C_1.0     C_2.0     C_4.0    C_10.0
solver_liblinear -0.116757 -0.116757 -0.116757 -0.116757 -0.116857 -0.116857 -0.119316 -0.119316 -0.120376 -0.120376
solver_saga      -0.121060 -0.121060 -0.121144 -0.121144 -0.121144 -0.121144 -0.121144 -0.121144 -0.121144 -0.121144


In [12]:
# Save pickle object
with open('./test/log_reg_gs_l1.pkl', 'wb') as f:
    pickle.dump(gs_cv_l1, f)

In [ ]:
# TODO: feature selection (low variance, correlated features, univariate feature selection, pca, ...) (https://towardsdatascience.com/feature-selection-and-dimensionality-reduction-f488d1a035de)

## Removing low variance features

In [20]:
from sklearn.feature_selection import VarianceThreshold

var_selector = VarianceThreshold()
X_removed = var_selector.fit_transform(X)
X_removed.shape, X.shape

((42766, 70), (42766, 70))

## Removing correlated features

In [76]:
corr_thresh = 0.6
corr_df = daily_df[feat_cols].corr()
corr_values = np.triu(corr_df.values, k=1)
high_corr_col_mask = np.any(np.abs(corr_values) > corr_thresh, axis=0)

high_corr_cols = corr_df.columns[high_corr_col_mask]
high_corr_cols

Index(['perceived trainingSuccess', 'perceived recovery', 'perceived trainingSuccess.1', 'perceived recovery.1', 'perceived trainingSuccess.2', 'perceived recovery.2', 'perceived trainingSuccess.3', 'perceived recovery.3', 'perceived trainingSuccess.4', 'perceived recovery.4', 'perceived trainingSuccess.5', 'perceived recovery.5', 'perceived trainingSuccess.6', 'perceived recovery.6'], dtype='object')

In [77]:
len(high_corr_cols)

14

In [53]:
# TODO: re-train linear and logistic regression

28